In [41]:
import pickle, warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

In [42]:
with open('../../data/processed/gdelt_intermediate_cleaned.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
df = df[['GKGRECORDID','article_title']]

In [ ]:
# Load model: https://huggingface.co/Qwen/Qwen3-Embedding-0.6B
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

In [45]:
prompt = "Instruct: Given a news headline, extract the sentiment that is most likely to affect the airline stock market\nQuery:"

In [ ]:
query_embeddings = model.encode(
    df['article_title'], 
    prompt=prompt,
    show_progress_bar=True,
    batch_size=32
    )

Batches: 100%|██████████| 157/157 [01:23<00:00,  1.87it/s]


In [ ]:
for i in range(query_embeddings.shape[1]):
    df[f'llm_dimension_{i:04d}'] = query_embeddings[:, i]

,GKGRECORDID,article_title,llm_dimension_0000,llm_dimension_0001,llm_dimension_0002,llm_dimension_0003,llm_dimension_0004,llm_dimension_0005,llm_dimension_0006,llm_dimension_0007,...,llm_dimension_1014,llm_dimension_1015,llm_dimension_1016,llm_dimension_1017,llm_dimension_1018,llm_dimension_1019,llm_dimension_1020,llm_dimension_1021,llm_dimension_1022,llm_dimension_1023
935947,20191212100000-1539,a pilot s perspective: pilot profile: bette ak...,0.052520,-0.019170,-0.015009,0.025224,0.002318,-0.022403,0.020386,0.033551,...,0.018646,-0.011007,-0.010550,0.004403,-0.039901,0.029571,-0.000511,0.024312,0.002538,0.033504
470108,20181211060000-1291,georgia hertz clear partner to speed rentals w...,0.019124,-0.037034,-0.011872,0.039794,0.005699,-0.012398,-0.033438,0.073490,...,0.000636,0.009283,-0.045080,-0.021422,-0.035820,-0.002073,-0.001423,0.009865,-0.013946,0.006115
310287,20241205234500-453,s&p 500 gains and losses today: synopsys sinks...,0.022685,0.014713,-0.008331,0.004574,0.040659,-0.003641,-0.028840,-0.011025,...,-0.003418,-0.010249,-0.016593,-0.003980,-0.018613,-0.004209,-0.033684,0.036132,0.026723,0.036909
533626,20210628094500-589,travelport adjusts fees and technology to navi...,0.042948,0.030596,-0.011718,0.023049,0.070838,-0.009056,0.001667,0.043347,...,-0.004175,-0.002077,-0.043296,0.004144,-0.032585,-0.007607,-0.001097,0.020318,0.000825,0.031432
176131,20230807201500-1460,conor mcgregor inserts himself in jake paul & ...,0.046629,-0.021889,-0.006276,0.032330,0.063353,-0.055479,-0.068951,-0.017145,...,-0.018371,0.011940,-0.039162,0.006347,-0.008012,-0.014109,0.011649,0.020408,-0.015854,0.027234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441657,20180111211500-655,meanwhile at walmart better pay better benefits,0.002983,-0.002308,-0.008743,0.043346,0.025134,-0.027202,-0.024256,0.017432,...,-0.017162,0.009336,-0.042711,-0.016227,-0.020188,0.020866,-0.022730,0.015213,-0.045402,0.032668
786648,20191118120000-1160,hillary clinton s plane in emergency u turn af...,0.028416,0.047371,-0.005647,0.043158,0.004018,-0.020675,-0.054257,0.035988,...,0.014187,-0.010218,-0.049020,0.027196,-0.037763,0.004098,-0.011946,-0.017346,0.013993,0.023978
633395,20190312011500-1287,u says 737 max safe fly ethiopia crash sector,0.033152,0.056928,-0.006072,0.069349,0.009029,-0.028968,-0.036609,0.056049,...,0.017686,-0.014565,-0.038315,0.015918,-0.015794,0.002052,-0.007793,-0.018119,0.021860,0.037178
599958,20210310211500-1331,hawaiian airlines starts long beach maui service,0.012484,0.014195,-0.008100,0.043807,0.013788,-0.037125,0.015531,0.043730,...,0.007577,-0.018525,-0.030896,-0.004105,-0.028558,0.006252,-0.006230,0.030611,0.001461,0.026684


In [ ]:
# Save df_sentiment to a pickle file
df.drop(columns=['article_title'], inplace=True)

with open('../../data/processed/gdelt_llm_sentiment.pkl', 'wb') as f:
    pickle.dump(df, f)